# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes.


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.

### First, download the data from: https://www.kaggle.com/datasets/ealaxi/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?
### Note: don't use the entire dataset, use a sample instead, with n=100000 elements, so your computer doesn't freeze.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [8]:
data = pd.read_csv("/content/drive/MyDrive/Labs Week 7/Lab5 Imbalance/PS_20174392719_1491204439457_log.csv").sample(n=100000, random_state=42)

In [11]:
data.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
3737323,278,CASH_IN,330218.42,C632336343,20866.00,351084.42,C834976624,452419.57,122201.15,0,0
264914,15,PAYMENT,11647.08,C1264712553,30370.00,18722.92,M215391829,0.00,0.00,0,0
85647,10,CASH_IN,152264.21,C1746846248,106589.00,258853.21,C1607284477,201303.01,49038.80,0,0
5899326,403,TRANSFER,1551760.63,C333676753,0.00,0.00,C1564353608,3198359.45,4750120.08,0,0
2544263,206,CASH_IN,78172.30,C813403091,2921331.58,2999503.88,C1091768874,415821.90,337649.60,0,0


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 3737323 to 6142173
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            100000 non-null  int64  
 1   type            100000 non-null  object 
 2   amount          100000 non-null  float64
 3   nameOrig        100000 non-null  object 
 4   oldbalanceOrg   100000 non-null  float64
 5   newbalanceOrig  100000 non-null  float64
 6   nameDest        100000 non-null  object 
 7   oldbalanceDest  100000 non-null  float64
 8   newbalanceDest  100000 non-null  float64
 9   isFraud         100000 non-null  int64  
 10  isFlaggedFraud  100000 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 9.2+ MB


In [10]:
display(data.describe())

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,100000.000000,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,100000.000000,100000.000000
mean,243.709070,1.805811e+05,8.366804e+05,8.582234e+05,1.104193e+06,1.230055e+06,0.001410,0.000010
std,142.518613,5.586699e+05,2.901104e+06,2.936799e+06,3.223011e+06,3.475326e+06,0.037524,0.003162
min,1.000000,9.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,156.000000,1.350821e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
50%,240.000000,7.603086e+04,1.393850e+04,0.000000e+00,1.387482e+05,2.185786e+05,0.000000,0.000000
75%,335.000000,2.091130e+05,1.070771e+05,1.464169e+05,9.605963e+05,1.126011e+06,0.000000,0.000000
max,736.000000,3.697390e+07,3.359321e+07,3.388709e+07,2.362896e+08,2.724047e+08,1.000000,1.000000


### What is the distribution of the outcome?

In [14]:
data['isFraud'].value_counts()

0    99859
1      141
Name: isFraud, dtype: int64

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [15]:
cleaned_data = data.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
cleaned_data['type'] = cleaned_data['type'].astype('category')
cleaned_data = pd.get_dummies(cleaned_data, columns=['type'], drop_first=True)

cleaned_data['step'] = pd.to_datetime(cleaned_data['step'], unit='s')

X = cleaned_data.drop('isFraud', axis=1)
y = cleaned_data['isFraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X_train.dtypes

step              datetime64[ns]
amount                   float64
oldbalanceOrg            float64
newbalanceOrig           float64
oldbalanceDest           float64
newbalanceDest           float64
type_CASH_OUT              uint8
type_DEBIT                 uint8
type_PAYMENT               uint8
type_TRANSFER              uint8
dtype: object

In [18]:
X_train['step'] = pd.to_datetime(X_train['step']).astype(int) // 10**9
X_test['step'] = pd.to_datetime(X_test['step']).astype(int) // 10**9

### Run a logisitc regression classifier and evaluate its accuracy.

In [20]:
lr_classifier = LogisticRegression(max_iter=1000)
lr_classifier.fit(X_train, y_train)
lr_pred = lr_classifier.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_pred)
print(lr_accuracy)
print(classification_report(y_test, lr_pred))

0.99865
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19972
           1       0.53      0.36      0.43        28

    accuracy                           1.00     20000
   macro avg       0.76      0.68      0.71     20000
weighted avg       1.00      1.00      1.00     20000



### Now pick a model of your choice and evaluate its accuracy.

In [22]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print( rf_accuracy)

0.99965


In [25]:
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19972
           1       1.00      0.75      0.86        28

    accuracy                           1.00     20000
   macro avg       1.00      0.88      0.93     20000
weighted avg       1.00      1.00      1.00     20000



### Which model worked better and how do you know?

** Logistic Regression worked better based on accuracy and classification report evaluation. This model has a higher accuracy score and its classification report shows better precision, recall, and F1-score values for both classes.**

### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.